In [12]:
import cv2
from cv2.ximgproc import guidedFilter
import matplotlib.pyplot as plt
import numpy as np

import os
import os.path as osp
from tqdm import tqdm

In [13]:
def get_filtered_map(img_path, dmap_path, dst_dir):
    
    # get the dmap file's name
    fname = os.path.split(dmap_path)[1]
    
    # read in the hazy image being used as the guide
    img = cv2.imread(img_path)/255
    
    # read in the depth map
    dmap = cv2.imread(dmap_path)/255
    
    # there might be a slight variation in the depth map shape 
    # due to the reconstruction by the model, so reshape it to 
    # image size
    dmap = cv2.resize(dmap, (img.shape[1], img.shape[0]))

    dmap = cv2.cvtColor(dmap.astype('float32'), cv2.COLOR_BGR2GRAY)
    tmap = np.clip(np.exp(-dmap), a_min=0, a_max=1)
    
    gray = cv2.cvtColor(img.astype('float32'), cv2.COLOR_BGR2GRAY)
    t = guidedFilter(gray, tmap.astype('float32'), radius=20, eps=10e-3)
    
    # write the refined dmap to a file
    cv2.imwrite(os.path.join(dst_dir, fname), t*255)

    return

In [14]:
dataset = "dh/NYU"
dmap_dir = osp.join("datasets", dataset, "dmap_pred")
hazy_dir = osp.join("datasets", dataset, "hazy")

dst_dir = osp.join("datasets", dataset, "dmap_pred_refined")
os.makedirs(dst_dir, exist_ok=True)

In [15]:
# accumulate file paths
hazy_paths, dmap_paths = [], []

for f in sorted(os.listdir(dmap_dir)):
    dmap_paths.append(os.path.join(dmap_dir, f))

for f in sorted(os.listdir(hazy_dir)):
    hazy_paths.append(os.path.join(hazy_dir, f))

In [16]:
# filter the dmaps using hazy images as guides

for guide_path, dmap_path in tqdm(zip(hazy_paths, dmap_paths), total=len(hazy_paths)):
    get_filtered_map(guide_path, dmap_path, dst_dir)

100%|██████████| 1449/1449 [00:43<00:00, 33.29it/s]
